- record a video for about 30 seconds
    - resolution: =, shutter speed: fastest, stabilization: turn off
    - covering the object in different angles. 
- drag and drop the video to the exe file.
- video -> MJPEG
- resolution : 1920*1080
- file format : MJPEG
- fps: 30fps


In [1]:
import cv2 as cv
import numpy as np

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

W=23
L=31

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((W*L, 3), np.float32)
objp[:,:2] = np.mgrid[0:L,0:W].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# imgpath = 'images/sample.jpg'
imgpath = 'images/camera_calib5.jpg'
img = cv.imread(imgpath)

grayscaledimg = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
cv.imshow(imgpath, grayscaledimg)
cv.waitKey(0)
ret, corners = cv.findChessboardCorners(grayscaledimg, (L,W), None)
print(ret)

if ret == True:
    objpoints.append(objp)

    corners2 = cv.cornerSubPix(grayscaledimg,corners, (11, 11), (-1, -1), criteria)
    imgpoints.append(corners2)

    # Draw and display the corners
    cv.drawChessboardCorners(img, (L, W), corners2, ret)
    window_name = imgpath + ' outputimg'
    cv.imshow(window_name, img)
    cv.waitKey(0)

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, grayscaledimg.shape[::-1], None, None)
    print(ret, mtx, dist, rvecs, tvecs)

True
0.2789343940438358 [[459.49627036   0.         462.98078721]
 [  0.         425.9391768  332.55095914]
 [  0.           0.           1.        ]] [[-0.13575777  0.0247265   0.00342986  0.00018729 -0.00074961]] (array([[-0.0740356 ],
       [ 0.00389051],
       [ 0.07810778]]),) (array([[-12.03832935],
       [-13.96143027],
       [ 20.05135273]]),)


## Getting Camera Matrix value from multiple images

In [4]:
import cv2 as cv
import numpy as np
import glob2 as glob

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

W=23
L=31

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((W*L, 3), np.float32)
objp[:,:2] = np.mgrid[0:L,0:W].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images_regex = 'images/camera_calib*.jpg'
images = glob.glob(images_regex) # recursively retrieving files with regex
 
for idx, fname  in enumerate(images):
    if idx == 0 or idx == len(images) - 1: print(idx)
    img = cv.imread(fname)
    grayscaledimg = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # cv.imshow(fname, grayscaledimg)
    # cv.waitKey(0)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(grayscaledimg, (31,23), None)
    # if idx == 0 or idx == len(images) - 1: 
    print(idx, ret) # 10, 19, 22, 24, 5
    # Question: some of the sample files returned True for the function fincChessboardCorners and some are not. 
    # -> https://stackoverflow.com/questions/66225558/cv2-findchessboardcorners-fails-to-find-corners
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
 
        corners2 = cv.cornerSubPix(grayscaledimg,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        cv.drawChessboardCorners(img, (31,23), corners2, ret)
        window_name = fname + ' outputimg'
        cv.imshow(window_name, img)
        # cv.waitKey(500)
        cv.waitKey(0)

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, grayscaledimg.shape[::-1], None, None)
print(mtx)
#cv.destroyAllWindows()

0
0 False
1 True
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 True
11 False
12 False
13 False
14 True
15 False
16 True
17 False
18 False
19 False
20 True
21 False
22 False
23 False
24
24 False
[[731.9983358    0.         450.94058942]
 [  0.         683.60570699 343.07428916]
 [  0.           0.           1.        ]]


## Getting Camera Matrix value from MJPEG file

In [ ]:
import cv2 as cv
import numpy as np

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

W=7
L=10 

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((W*L, 3), np.float32)
objp[:,:2] = np.mgrid[0:L, 0:W].T.reshape(-1, 2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# preparting videoio registry 
availableBackends = [cv.videoio_registry.getBackendName(b) for b in cv.videoio_registry.getBackends()]
print(availableBackends)

videopath = 'input/test3.mjpg'
cap = cv.VideoCapture(videopath, cv.CAP_OPENCV_MJPEG) # capturing a video file. after capturing -> can read the frames onr by one.
print(cap)
# value of the constants: https://docs.opencv.org/4.3.0/d4/d15/group__videoio__flags__base.html ;  stored using enum. 

while (cap.isOpened()):
    ret, frame = cap.read()  # read the next frame of the video. 
    print("Frame available exist:", ret)

    if ret:
        img = cv.imread(frame) # read the frame as a image. 
        cv.imshow('frame read', img)
        cv.waitKey(0)
        grayscaledimg = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(grayscaledimg, (L, W), None)
        print(idx, ret) 

        if ret == True:
            objpoints.append(objp)
    
            corners2 = cv.cornerSubPix(grayscaledimg, corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)
    
            # Draw and display the corners
            cv.drawChessboardCorners(img, (L, W), corners2, ret)

        ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, grayscaledimg.shape[::-1], None, None)
        print(mtx)
    else: # all the frames are grabbed and there's no remaining frame of the video. 
        break

cap.release() # closing a video file



['FFMPEG', 'GSTREAMER', 'INTEL_MFX', 'MSMF', 'MSMF', 'DSHOW', 'CV_IMAGES', 'CV_MJPEG', 'UEYE', 'OBSENSOR']
< cv2.VideoCapture 000002705B171430>
Frame available exist: True


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
